# SABR Volatility Surface MDA-CNN Tutorial

This notebook demonstrates the complete workflow for SABR volatility surface modeling using Multi-fidelity Data Aggregation CNN (MDA-CNN).

## Overview

The SABR MDA-CNN system combines:
- **High-fidelity Monte Carlo simulations** for accurate volatility surfaces
- **Low-fidelity Hagan analytical approximations** for fast surface generation
- **MDA-CNN architecture** that learns residuals between MC and Hagan surfaces

This approach achieves high accuracy with minimal expensive Monte Carlo data points.

## Setup and Imports

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Project imports
from utils.config import ConfigManager
from utils.logging_utils import setup_logging
from data_generation.sabr_params import SABRParams, GridConfig
from data_generation.sabr_mc_generator import SABRMCGenerator
from data_generation.hagan_surface_generator import HaganSurfaceGenerator
from models.mda_cnn import create_mda_cnn_model
from visualization.smile_plotter import SmilePlotter, create_smile_data
from visualization.surface_plotter import SurfacePlotter

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("Set1")
%matplotlib inline

# Set up logging
setup_logging(level='INFO')

print("Setup complete!")

## 1. Data Generation

First, let's generate SABR volatility surfaces using both Monte Carlo and Hagan methods.

In [ ]:
# Define SABR parameters
sabr_params = SABRParams(
    forward=100.0,
    alpha=0.3,
    beta=0.7,
    nu=0.4,
    rho=-0.3
)

# Define grid configuration
grid_config = GridConfig(
    strike_range=(70.0, 130.0),
    maturity_range=(0.1, 2.0),
    n_strikes=20,
    n_maturities=10
)

print(f"SABR Parameters: {sabr_params}")
print(f"Grid Configuration: {grid_config}")
print(f"Total grid points: {grid_config.n_strikes * grid_config.n_maturities}")

In [ ]:
# Generate high-fidelity Monte Carlo surface
print("Generating Monte Carlo surface...")
mc_generator = SABRMCGenerator(n_paths=50000, random_seed=42)
hf_surface = mc_generator.generate_surface(sabr_params, grid_config)

# Generate low-fidelity Hagan surface
print("Generating Hagan analytical surface...")
hagan_generator = HaganSurfaceGenerator()
lf_surface = hagan_generator.generate_surface(sabr_params, grid_config)

# Calculate residuals
residuals = hf_surface - lf_surface

print(f"HF surface shape: {hf_surface.shape}")
print(f"LF surface shape: {lf_surface.shape}")
print(f"Residual statistics:")
print(f"  Mean: {np.mean(residuals):.6f}")
print(f"  Std:  {np.std(residuals):.6f}")
print(f"  Max:  {np.max(np.abs(residuals)):.6f}")

## Command Line Interface Preview

Here's how you would use the command-line scripts:

In [ ]:
# Command line examples (for reference)
commands = [
    "# Generate training data",
    "python generate_data.py --n-surfaces 1000 --hf-budget 200 --output-dir data/experiment1",
    "",
    "# Train MDA-CNN model", 
    "python train_model.py --data-dir data/experiment1 --model mda_cnn --epochs 100",
    "",
    "# Run HF budget analysis",
    "python train_model.py --data-dir data/experiment1 --hf-budget-analysis --budgets 50 100 200 500",
    "",
    "# Evaluate results",
    "python evaluate_model.py --results-dir results/training_20241201_120000 --comprehensive-analysis",
    "",
    "# Generate visualizations",
    "python evaluate_model.py --results-dir results/ --visualize --interactive"
]

print("Command Line Usage Examples:")
print("=" * 40)
for cmd in commands:
    print(cmd)